In [1]:
# geospatial analysis
import geopandas as gpd

In [3]:
# essential variables

# let's try to convert everything to EPSG:5070. For more accurate area measurements

# boundary for state of California
us_states = gpd.read_file("Data/Boundaries/cb_2018_us_state_500k/cb_2018_us_state_500k.shp")
us_states.to_crs("EPSG:5070", inplace=True)
ca_state = us_states[us_states["STUSPS"] == "CA"]

# HUC8 subbasins
huc8 = gpd.read_file("Data/Boundaries/HUC8_CONUS/HUC8_US.shp")
huc8.to_crs("EPSG:5070", inplace=True)
huc8['CA'] = huc8["STATES"].map(lambda x: "CA" in x)
huc8 = huc8[huc8["CA"]]

# intersect with California
huc8_ca = gpd.clip(huc8, ca_state)

### Land Restrictions

In [ ]:
# let's not worry about utilities for now, just for time. and I don't think it's the most important restriction
# # utilities map in California
# ca_utilities = gpd.read_file("Data/Boundaries/ElectricLoadServingEntities_IOU_POU_-6992589901327565970.geojson")

In [ ]:
# federal, state, local lands
# not too many tribal lands in California so we don't worry about that

# overall takes 30 seconds
us_lands = gpd.read_file("Data/Boundaries/USA_Federal_Lands_1879006603790792540.geojson")

# filter invalid geometries
us_lands = us_lands[~us_lands['geometry'].isna()]
us_lands = us_lands[us_lands['geometry'].is_valid]

us_lands.to_crs("EPSG:5070", inplace=True)

# first filter US lands to California
ca_geometry = ca_state.iloc[0].geometry
us_lands_ca = us_lands[us_lands.intersects(ca_geometry)]

In [31]:
# first filter US lands to California
ca_geometry = ca_state.iloc[0].geometry

# for idx, row in us_lands.iterrows():
#     try:
#         row['geometry'].intersects(ca_geometry)
#     except:
#         print(idx)
us_lands.intersects(ca_geometry).value_counts()


# all_us_land = us_lands.union_all(method="coverage")

False    163123
True      10309
Name: count, dtype: int64

In [ ]:
# cut out 

subbasin_us_land = []

for subbasin in huc8_ca:
    if all_us_land.contains(subbasin):
        subbasin_us_land.append(True)
    else:
        subbasin_us_land.append(False)

huc8_ca["Government_Land"] = subbasin_us_land
huc8_ca = huc8_ca[~huc8_ca["Government Land"]]

,OBJECTID,Agency,unit_name,link,GlobalID,geometry
0,1,National Park Service,Hopewell Culture National Historical Park,https://www.nps.gov/hocu,3eae02d9-96db-46b8-be06-620ab3149938,"POLYGON ((-83.00689 39.37362, -83.00716 39.373..."
1,2,National Park Service,Hopewell Culture National Historical Park,https://www.nps.gov/hocu,ab6ddbc1-80ae-48a3-b2f1-67459045c0cf,"POLYGON ((-82.92295 39.286, -82.92308 39.28589..."
2,3,National Park Service,Hopewell Culture National Historical Park,https://www.nps.gov/hocu,64f5d5ba-3cb0-4386-ae8f-ead1d9b6fad7,"POLYGON ((-83.09336 39.36848, -83.08853 39.365..."
3,4,National Park Service,Hopewell Culture National Historical Park,https://www.nps.gov/hocu,12bb7bf9-56d1-4cc4-bf50-083c6ca198f5,"POLYGON ((-83.21771 39.24333, -83.21754 39.243..."
4,5,National Park Service,Hopewell Culture National Historical Park,https://www.nps.gov/hocu,36ead365-68d4-4d44-bbd7-880720528156,"POLYGON ((-82.9771 39.38838, -82.9756 39.38552..."
...,...,...,...,...,...,...
176090,176091,Fish and Wildlife Service,Nobles County Waterfowl Production Area,https://www.fws.gov/,8ed8036f-2908-4ad6-b14d-724b339ca7d1,"POLYGON ((-95.61425 43.52173, -95.61424 43.519..."
176091,176092,Fish and Wildlife Service,Nobles County Waterfowl Production Area,https://www.fws.gov/,3b60b494-f93a-447b-bab9-a469de3ac839,"POLYGON ((-95.62028 43.51452, -95.62028 43.514..."
176092,176093,Fish and Wildlife Service,Nobles County Waterfowl Production Area,https://www.fws.gov/,db939826-b929-4f70-b783-cabd36b3cf48,"POLYGON ((-95.58014 43.62571, -95.58012 43.627..."
176093,176094,Fish and Wildlife Service,Watonwan County Waterfowl Production Area,https://www.fws.gov/,b9cdfde5-f6b4-4549-ac14-1b7a55236198,"POLYGON ((-94.75274 44.08781, -94.75112 44.091..."
